In [1]:


from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_similarity,
)


c:\Users\SSAFY\Desktop\project_2\S12P21A702\finafanAI\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 평가할 메트릭 리스트
metrics = [
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
    answer_similarity
]


In [ ]:
import asyncio
from app.service.v1.search import fast_news_search
from app.service.v1.prompts import idol_news_prompt
from langchain_openai import ChatOpenAI
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
)
from datasets import Dataset  # ✅ 핵심
import pandas as pd
import matplotlib.pyplot as plt

# ✅ 질문 리스트
sample_questions = [
    "이찬원 요즘 뭐해?",
    "임영웅 요즘 뭐해?",
    "영웅이 소식 알려줘",
    "임영웅 뉴스 찾아줘",
    "정동원 뉴스 찾아",
    "박지민 최근 기사 있어?",
    "방탄소년단 관련 뉴스 보여줘",
    "아이유 뉴스 뭐 떴어?",
    "이찬원 관련 기사 찾아줘",
    "장원영 근황 기사 알려줘",
    "지민이 무슨 뉴스 있어?",
    "정국 기사 좀 줘",
    "수지 요즘 뭐하고 있어?",
    "임영웅 기사 뭐 나왔어?",
    "이찬원 근황 알려줘",
    "아이브 뉴스 보여줘",
    "엔하이픈 관련 뉴스 있을까?",
    "김세정 최근 기사 뭐 있어?",
    "에스파 뉴스 보여줘",
    "르세라핌 기사 좀 찾아줘"
]

# ✅ 체인 구성
llm = ChatOpenAI(temperature=0.1)
idol_news_chain = idol_news_prompt | llm

# ✅ 평가용 샘플 생성 함수
async def build_ragas_sample(question: str):
    try:
        docs = fast_news_search(question) or ["(관련 기사가 없습니다.)"]
        contexts = docs[:5]

        input_data = {
            "topic": question,
            "results": docs
        }

        response = await idol_news_chain.ainvoke(input_data)

        return {
            "question": question,
            "contexts": contexts if isinstance(contexts, list) else [str(contexts)],
            "answer": response.content.strip(),
            "ground_truth": None
        }

    except Exception as e:
        print(f"❌ [{question}] 처리 중 오류: {e}")
        return None


# ✅ 평가 루프
async def run_ragas_evaluation():
    print("🚀 뉴스 기반 체인 평가 시작...")
    samples = await asyncio.gather(*[build_ragas_sample(q) for q in sample_questions])
    samples = [s for s in samples if s]

    if not samples:
        print("⚠️ 유효한 샘플이 없습니다. 평가를 종료합니다.")
        return

    # ✅ HuggingFace Dataset으로 변환
    df = pd.DataFrame(samples)
    ragas_dataset = Dataset.from_pandas(df)

    print(f"\n✅ 총 {len(ragas_dataset)}개 샘플 평가 준비 완료")

    # ✅ 평가 실행
# ✅ 평가 실행 (ground_truth 없이 평가 가능한 메트릭만 사용)
    results = evaluate(
        ragas_dataset,
        metrics=[
        faithfulness,
        answer_relevancy
        # context_precision,
        # context_recall
        ]
    )


    print("\n📊 RAGAS 평가 결과:")
    print(results)

    # ✅ 그래프 출력
    results.plot(kind='bar', figsize=(10, 5), title="RAGAS Evaluation Metrics", ylim=(0, 1))
    plt.ylabel("Score")
    plt.xticks(rotation=45)
    plt.grid(axis="y", linestyle="--")
    plt.tight_layout()
    plt.show()

    # ✅ CSV 저장
    df.to_csv("idol_news_ragas_dataset.csv", index=False)
    results.to_csv("idol_news_ragas_metrics.csv")



ModuleNotFoundError: No module named 'app'

In [ ]:
asyncio.run(run_ragas_evaluation())